In [ ]:
#this is needed for expand string constraction like can't -> can not,won't-> will not
!pip install contractions
#this is needed for convert text to number like three -> 3
!pip install text2num

     |████████████████████████████████| 245kB 3.7MB/s 
     |████████████████████████████████| 317kB 28.9MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81701 sha256=15881500bc70e404f9bc91f8fc28c258a8103156be338bf71fb29e26712e5d75
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick
  Created wheel for text2num: filename=text2num-2.2.1-cp36-none-any.whl size=27338 sha256=a845bffc33751e52dbbdf3bdec084c23b3d0bcb1b853497fc6ef69feb9306e5d
  Stored in directory: /root/.cache/pip/wheels/44/df/80/e253ea78ba47fd987f259e301be3fa60352ebb932e575fb118
Successfully built text2num


In [ ]:
#import library
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
import json
import string
import contractions
from textblob import TextBlob
import unidecode
from text_to_num import alpha2digit
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer

#Read dataset 

In [ ]:
#Read dataset
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
dataset = pd.read_csv('gdrive/My Drive/Abusive_text_detection/Dataset_collection/final_hatespeechtwitter.csv')
# print(dataset)

# Preprocessing start(1 - 14)

In [ ]:
#1
def convert_lower(text):
  return text.lower()

In [ ]:
#2
def accented_char_to_ascii_char(text):
  return unidecode.unidecode(text)

In [ ]:
#3
def expand_contraction(text):
  return contractions.fix(text)

In [ ]:
#4
def number_words_to_numeric_form(text):
  return alpha2digit(text, "en")

In [ ]:
#5
def remove_urls(text):
  return re.sub(r"(http|https)(\s*:\s*)(\s*//\s*)\S+|www\.\S+","",text)

In [ ]:
#6
def remove_email(text):
  return re.sub(r"([a-zA-Z0-9+._-]+(\s*@\s*)[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)","",text)

In [ ]:
#7
def remove_hashtag_and_usertag(text):
  return re.sub(r"#\s*\S+|@\s*\S+","",text)

In [ ]:
#8
def remove_rt_url_different_numeric_tag_numbers(text):
  return re.sub(r'rt|url|(\w*\d\w*)', '', text)

In [ ]:
#9
def remove_punctuation(text):
  return re.sub('[%s]' % re.escape(string.punctuation), '', text)

In [ ]:
#10
def remove_extra_white_spaces(text):
  return ' '.join(text.split())

In [ ]:
#11
def spelling_correction(text):
  return str(TextBlob(text).correct())

In [ ]:
#12
def lemmatization(text):
  return ' '.join([WordNetLemmatizer().lemmatize(word) for word in text.split()])

In [ ]:
#13
def stemming(text):
  return ' '.join([PorterStemmer().stem(word) for word in text.split()])

In [ ]:
#14
def remove_stopword(text):
  return ' '.join([word for word in text.split() if word not in stopwords.words('english')])

In [ ]:
def cleaning(dataset):
  corpus = list(dataset['Tweets'])
  # print(corpus)
  for i,m in enumerate(corpus):
    text = convert_lower(m)                                       #1
    text = accented_char_to_ascii_char(text)                      #2
    text = expand_contraction(text)                               #3
    text = number_words_to_numeric_form(text)                     #4
    text = remove_urls(text)                                      #5
    text = remove_email(text)                                     #6
    text = remove_hashtag_and_usertag(text)                       #7
    text = remove_rt_url_different_numeric_tag_numbers(text)      #8
    text = remove_punctuation(text)                               #9
    text = remove_extra_white_spaces(text)                        #10
    # text = spelling_correction(text)                              #11
    text = lemmatization(text)                                    #12
    # text = stemming(text)                                         #13
    text = remove_stopword(text)                                  #14
    corpus[i] = text
    # print('no: ',i)
  return corpus

In [ ]:
corpus = cleaning(dataset)
# corpus = np.array(corpus)
# print(corpus)

In [ ]:
#Making json file of preprocessed tweets
corpus_file = open("final_data.json", "w")  
json.dump(corpus, corpus_file, indent = 6)  
corpus_file.close()

In [ ]:
###for saving the file in google drive
!cp final_data.json "/content/gdrive/My Drive/Abusive_text_detection/Preprocess"

In [ ]:
#Making json file of preprocessed tweet's Labels
label_file = open("class_value.json", "w")  
json.dump(list(dataset['abusive']), label_file, indent = 6)  
label_file.close()

In [ ]:
###for saving the file in google drive
!cp class_value.json "/content/gdrive/My Drive/Abusive_text_detection/Preprocess"